# Cubic with discretized derivatives

Using 4 points for the interpolation

In [6]:
using SymPy

@syms x Δ δ
xs = [
    symbols("x_{i-1}"),
    symbols("x_{i}"),
    symbols("x_{i+1}"),
    symbols("x_{i+2}"),
]
xs = [i*Δ for i in -1:2]
ys = [
    symbols("y_{i-1}"),
    symbols("y_{i}"),
    symbols("y_{i+1}"),
    symbols("y_{i+2}"),
]
cs = [symbols("c_{$i}") for i in 0:3]


4-element Vector{Sym}:
 c_{0}
 c_{1}
 c_{2}
 c_{3}

## General case

In [7]:
cubic_pol = sum(symbols("c_{$i}")*x^i for i in 0:3)
eqs = [
    cubic_pol.subs(x,xs[2] - xs[2]) - ys[2], # f(0)
    cubic_pol.subs(x,xs[3] - xs[2]) - ys[3], # f(Δ)
    cubic_pol.diff(x).subs(x,xs[2] - xs[2]) - (ys[3] - ys[1])/(xs[3] - xs[1]),# f'(0)
    cubic_pol.diff(x).subs(x,xs[3] - xs[2]) - (ys[4] - ys[2])/(xs[4] - xs[2]) # f'(Δ)
]
eqs = expand.(eqs)

sol = solve(eqs,cs)
appr = cubic_pol.subs(sol) |> expand

for y in ys
    appr.coeff(y).subs(x/Δ,δ) |> display
end

   3         
  δ     2   δ
- ── + δ  - ─
  2         2

   3      2    
3⋅δ    5⋅δ     
──── - ──── + 1
 2      2      

     3           
  3⋅δ       2   δ
- ──── + 2⋅δ  + ─
   2            2

 3    2
δ    δ 
── - ──
2    2 

In [8]:
eqs

4-element Vector{Sym}:
                                                 c_{0} - y_{i}
             c_{0} + c_{1}*Δ + c_{2}*Δ^2 + c_{3}*Δ^3 - y_{i+1}
                         c_{1} - y_{i+1}/(2*Δ) + y_{i-1}/(2*Δ)
 c_{1} + 2*c_{2}*Δ + 3*c_{3}*Δ^2 - y_{i+2}/(2*Δ) + y_{i}/(2*Δ)

## Near endpoints

### Beginning

In [11]:
##Beginning Not enforcing unknown conditions
quad_pol = sum(symbols("c_{$i}")*x^i for i in 0:2)
eqs = [
    cubic_pol.subs(x,xs[2] - xs[2]) - ys[2], # f(0)
    cubic_pol.subs(x,xs[3] - xs[2]) - ys[3], # f(Δ)
    cubic_pol.diff(x).subs(x,xs[2] - xs[2]) - (ys[3] - ys[2])/(xs[3] - xs[2]), # f'(Δ)
    cubic_pol.diff(x).subs(x,xs[3] - xs[2]) - (ys[4] - ys[2])/(xs[4] - xs[2]) # f'(Δ)
]
eqs = expand.(eqs)

sol = solve(eqs,cs)
appr = cubic_pol.subs(sol) |> expand

for y in ys
    appr.coeff(y).subs(x/Δ,δ) |> display
end

0

 3    2        
δ    δ         
── - ── - δ + 1
2    2         

   3    2    
- δ  + δ  + δ

 3    2
δ    δ 
── - ──
2    2 

### End

In [179]:
##End Not enforcing unknown conditions
quad_pol = sum(symbols("c_{$i}")*x^i for i in 0:2)

eqs = [
    quad_pol.subs(x,xs[2] - xs[2]) - ys[2], # f(0)
    quad_pol.subs(x,xs[3] - xs[2]) - ys[3], # f(Δ)
    quad_pol.diff(x).subs(x,xs[2] - xs[2]) - (ys[3] - ys[1])/((xs[3] - xs[1])),# f'(0)
    quad_pol.diff(x).subs(x,xs[3] - xs[2]) - (ys[3] - ys[2])/(xs[3] - xs[2]) # f'(Δ)

]
eqs = expand.(eqs)

sol = solve(eqs,cs)
appr = quad_pol.subs(sol) |> expand

for y in ys
    appr.coeff(y).subs(x/Δ,δ) |> display
end

 2    
δ    δ
── - ─
2    2

     2
1 - δ 

 2    
δ    δ
── + ─
2    2

0

# Cubic with Lagrangian polynomials

In [38]:
using SymPy

@syms x Δ δ

(x, Δ, δ)

In [40]:
idxs = -1:2
xs = [i*Δ for i in idxs]
ys = [symbols("y_{$i}") for i in idxs]

4-element Vector{Sym}:
 y_{-1}
  y_{0}
  y_{1}
  y_{2}

In [42]:
p(j,xs = xs) = prod((x-xs[k])/(xs[j]-xs[k]) for k in 1:length(xs) if k != j)

p (generic function with 2 methods)

In [43]:
for i in 1:length(xs)
    p(i).expand().subs(x/Δ, δ) |> display
end

   3    2    
  δ    δ    δ
- ── + ── - ─
  6    2    3

 3             
δ     2   δ    
── - δ  - ─ + 1
2         2    

   3    2    
  δ    δ     
- ── + ── + δ
  2    2     

 3    
δ    δ
── - ─
6    6

## Connecting piecewise LP-s

In [52]:
idxs = -1:2
xs0 = [i*Δ for i in idxs]
ys0 = [symbols("y_{$i}") for i in idxs]

idxs = -0:3
xs1 = [i*Δ for i in idxs]
ys1 = [symbols("y_{$i}") for i in idxs]

4-element Vector{Sym}:
 y_{0}
 y_{1}
 y_{2}
 y_{3}

In [64]:
pol0 = sum(y*p(j,xs0) for (j,y) in enumerate(ys0)).expand()#.subs(x/Δ, δ)
pol1 = sum(y*p(j,xs1) for (j,y) in enumerate(ys1)).expand()#.subs(x/Δ, δ);
display(pol0)
display(pol1)

   3           3          3          3          2           2          2      
  x ⋅y_{-1}   x ⋅y_{0}   x ⋅y_{1}   x ⋅y_{2}   x ⋅y_{-1}   x ⋅y_{0}   x ⋅y_{1}
- ───────── + ──────── - ──────── + ──────── + ───────── - ──────── + ────────
        3          3          3          3           2         2           2  
     6⋅Δ        2⋅Δ        2⋅Δ        6⋅Δ         2⋅Δ         Δ         2⋅Δ   

                                                 
   x⋅y_{-1}   x⋅y_{0}   x⋅y_{1}   x⋅y_{2}        
 - ──────── - ─────── + ─────── - ─────── + y_{0}
     3⋅Δ        2⋅Δ        Δ        6⋅Δ          
                                                 

   3          3          3          3          2            2            2    
  x ⋅y_{0}   x ⋅y_{1}   x ⋅y_{2}   x ⋅y_{3}   x ⋅y_{0}   5⋅x ⋅y_{1}   2⋅x ⋅y_{
- ──────── + ──────── - ──────── + ──────── + ──────── - ────────── + ────────
       3          3          3          3         2            2           2  
    6⋅Δ        2⋅Δ        2⋅Δ        6⋅Δ         Δ          2⋅Δ           Δ   

      2                                                             
2}   x ⋅y_{3}   11⋅x⋅y_{0}   3⋅x⋅y_{1}   3⋅x⋅y_{2}   x⋅y_{3}        
── - ──────── - ────────── + ───────── - ───────── + ─────── + y_{0}
          2        6⋅Δ           Δ          2⋅Δ        3⋅Δ          
       2⋅Δ                                                          

In [65]:
pol0.subs(x,Δ) - pol1.subs(x,Δ)

0

In [66]:
pol0.diff(x).subs(x,Δ) - pol1.diff(x).subs(x,Δ)

y_{-1}   2⋅y_{0}   y_{1}   2⋅y_{2}   y_{3}
────── - ─────── + ───── - ─────── + ─────
 6⋅Δ       3⋅Δ       Δ       3⋅Δ      6⋅Δ 

In [67]:
pol0.diff(x,2).subs(x,Δ) - pol1.diff(x,2).subs(x,Δ)

0

In [59]:
xs0|>display
xs1|>display

4-element Vector{Sym}:
  -Δ
   0
   Δ
 2⋅Δ

4-element Vector{Sym}:
   0
   Δ
 2⋅Δ
 3⋅Δ

# Fifth order polynomial interpolation

Using 6 points for the interpolation

In [46]:
using SymPy

@syms x Δ δ
xs = [i*Δ for i in -2:3]
ys = [
    symbols("y_{i-2}"),
    symbols("y_{i-1}"),
    symbols("y_{i}"),
    symbols("y_{i+1}"),
    symbols("y_{i+2}"),
    symbols("y_{i+3}"),
]
cs = [symbols("c_{$i}") for i in 0:5]

6-element Vector{Sym}:
 c_{0}
 c_{1}
 c_{2}
 c_{3}
 c_{4}
 c_{5}

In [48]:
disc_diff(i, ys = ys) = (ys[i-2] - 8*ys[i-1] + 8*ys[i+1] - ys[i+2])/(12Δ)
disc_diff2(i, ys = ys) = (-ys[i-2]+16*ys[i-1]-30*ys[i]+16*ys[i+1]-ys[i+2])/(12*Δ^2)

disc_diff2 (generic function with 2 methods)

In [49]:
i1, i2 = 3,4
quintic_pol = sum(symbols("c_{$i}")*x^i for i in 0:5)
eqs = [
    quintic_pol.subs(x,xs[i1]) - ys[i1], # f(0)
    quintic_pol.subs(x,xs[i2]) - ys[i2], # f(Δ)
    quintic_pol.diff(x).subs(x,xs[i1]) - disc_diff(i1,ys),# f'(0)
    quintic_pol.diff(x).subs(x,xs[i2]) - disc_diff(i2,ys), # f'(Δ)
    quintic_pol.diff(x,2).subs(x,xs[i1]) - disc_diff2(i1,ys),# f''(0)
    quintic_pol.diff(x,2).subs(x,xs[i2]) - disc_diff2(i2,ys), # f''(Δ)
]
eqs = expand.(eqs)

sol = solve(eqs,cs)
appr = quintic_pol.subs(sol) |> expand

for y in ys
    appr.coeff(y).subs(x/Δ,δ) |> display
end

     5       4      3    2     
  5⋅δ    13⋅δ    3⋅δ    δ    δ 
- ──── + ───── - ──── - ── + ──
   24      24     8     24   12

    5      4       3      2      
25⋅δ    8⋅δ    13⋅δ    2⋅δ    2⋅δ
───── - ──── + ───── + ──── - ───
  24     3       8      3      3 

      5       4       3      2    
  25⋅δ    21⋅δ    35⋅δ    5⋅δ     
- ───── + ───── - ───── - ──── + 1
    12      4       12     4      

    5       4       3      2      
25⋅δ    31⋅δ    11⋅δ    2⋅δ    2⋅δ
───── - ───── + ───── + ──── + ───
  12      6       4      3      3 

      5       4       3    2     
  25⋅δ    61⋅δ    11⋅δ    δ    δ 
- ───── + ───── - ───── - ── - ──
    24      24      8     24   12

   5    4      3
5⋅δ    δ    7⋅δ 
──── - ── + ────
 24    2     24 

### Left side

#### 1 point missing ($x_{-1} = x_{min}$)

In [50]:
disc_diff_a(i, ys = ys) = (- 2*ys[i-1] - 3*ys[i] +6*ys[i+1] - ys[i+2])/(6Δ)
disc_diff2_a(i, ys = ys) = (ys[i-1]-2*ys[i]+ys[i+1])/(Δ^2)
i1, i2 = 3,4

eqs = [
    quintic_pol.subs(x,xs[i1]) - ys[i1], # f(0)
    quintic_pol.subs(x,xs[i2]) - ys[i2], # f(Δ)
    quintic_pol.diff(x).subs(x,xs[i1]) - disc_diff_a(i1,ys),# f'(0)
    quintic_pol.diff(x).subs(x,xs[i2]) - disc_diff(i2,ys), # f'(Δ)
    quintic_pol.diff(x,2).subs(x,xs[i1]) - disc_diff2_a(i1,ys),# f''(0)
    quintic_pol.diff(x,2).subs(x,xs[i2]) - disc_diff2(i2,ys), # f''(Δ)
]
eqs = expand.(eqs)

sol = solve(eqs,cs)
appr = quintic_pol.subs(sol) |> expand

for y in ys
    appr.coeff(y).subs(x/Δ,δ) |> display
end

0

   5    4    3    2    
5⋅δ    δ    δ    δ    δ
──── - ── + ── + ── - ─
 24    2    8    2    3

     5             3             
  5⋅δ       4   2⋅δ     2   δ    
- ──── + 2⋅δ  - ──── - δ  - ─ + 1
   6             3          2    

   5             3    2    
5⋅δ       4   5⋅δ    δ     
──── - 3⋅δ  + ──── + ── + δ
 4             4     2     

     5                
  5⋅δ       4    3   δ
- ──── + 2⋅δ  - δ  - ─
   6                 6

   5    4      3
5⋅δ    δ    7⋅δ 
──── - ── + ────
 24    2     24 

#### 2 points missing ($x_{0} = x_{min}$)

In [51]:
disc_diff_b(i, ys = ys) = (- 3*ys[i] + 4*ys[i+1] - ys[i+2])/(2Δ)
disc_diff2_b(i, ys = ys) = (ys[i]-2*ys[i+1]+ys[i+2])/(Δ^2)
i1, i2 = 3,4

eqs = [
    quintic_pol.subs(x,xs[i1]) - ys[i1], # f(0)
    quintic_pol.subs(x,xs[i2]) - ys[i2], # f(Δ)
    quintic_pol.diff(x).subs(x,xs[i1]) - disc_diff_b(i1,ys),# f'(0)
    quintic_pol.diff(x).subs(x,xs[i2]) - disc_diff_a(i2,ys), # f'(Δ)
    quintic_pol.diff(x,2).subs(x,xs[i1]) - disc_diff2_b(i1,ys),# f''(0)
    quintic_pol.diff(x,2).subs(x,xs[i2]) - disc_diff2_a(i2,ys), # f''(Δ)
]
eqs = expand.(eqs)

sol = solve(eqs,cs)
appr = quintic_pol.subs(sol) |> expand

for y in ys
    appr.coeff(y).subs(x/Δ,δ) |> display
end

0

0

   5      4      3    2          
  δ    7⋅δ    2⋅δ    δ    3⋅δ    
- ── + ──── - ──── + ── - ─── + 1
  2     6      3     2     2     

   5      4                  
3⋅δ    7⋅δ       3    2      
──── - ──── + 2⋅δ  - δ  + 2⋅δ
 2      2                    

     5      4           2    
  3⋅δ    7⋅δ       3   δ    δ
- ──── + ──── - 2⋅δ  + ── - ─
   2      2            2    2

 5      4      3
δ    7⋅δ    2⋅δ 
── - ──── + ────
2     6      3  

### Right side

#### 1 point missing ($x_{2} = x_{max}$)

In [65]:
disc_diff_d(i1)

-4⋅y_{i-1} + y_{i-2} + 3⋅y_{i}
──────────────────────────────
             2⋅Δ              

In [59]:
disc_diff2(i1)

16⋅y_{i+1} - y_{i+2} + 16⋅y_{i-1} - y_{i-2} - 30⋅y_{i}
──────────────────────────────────────────────────────
                            2                         
                        12⋅Δ                          

In [73]:
disc_diff_c(i, ys = ys) = (ys[i-2] - 6*ys[i-1] + 3*ys[i] + 2*ys[i+1])/(6Δ)
disc_diff2_c(i, ys = ys) = (ys[i-1]-2*ys[i]+ys[i+1])/(Δ^2)
i1, i2 = 3,4

eqs = [
    quintic_pol.subs(x,xs[i1]) - ys[i1], # f(0)
    quintic_pol.subs(x,xs[i2]) - ys[i2], # f(Δ)
    quintic_pol.diff(x).subs(x,xs[i1]) - disc_diff(i1,ys),# f'(0)
    quintic_pol.diff(x).subs(x,xs[i2]) - disc_diff_c(i2,ys), # f'(Δ)
    quintic_pol.diff(x,2).subs(x,xs[i1]) - disc_diff2(i1,ys),# f''(0)
    quintic_pol.diff(x,2).subs(x,xs[i2]) - disc_diff2_c(i2,ys), # f''(Δ)
]
eqs = expand.(eqs)

sol = solve(eqs,cs)
appr = quintic_pol.subs(sol) |> expand

for y in ys
    appr.coeff(y).subs(x/Δ,δ) |> display
end

     5       4      3    2     
  5⋅δ    13⋅δ    3⋅δ    δ    δ 
- ──── + ───── - ──── - ── + ──
   24      24     8     24   12

   5       4      3      2      
5⋅δ    13⋅δ    4⋅δ    2⋅δ    2⋅δ
──── - ───── + ──── + ──── - ───
 6       6      3      3      3 

     5       4      3      2    
  5⋅δ    13⋅δ    7⋅δ    5⋅δ     
- ──── + ───── - ──── - ──── + 1
   4       4      4      4      

   5       4           2      
5⋅δ    13⋅δ     3   2⋅δ    2⋅δ
──── - ───── + δ  + ──── + ───
 6       6           3      3 

     5       4      3    2     
  5⋅δ    13⋅δ    5⋅δ    δ    δ 
- ──── + ───── - ──── - ── - ──
   24      24     24    24   12

0

#### 2 points missing ($x_{1} = x_{max}$)

In [74]:
disc_diff_d(i, ys = ys) = (1*ys[i-2] - 4*ys[i-1] + 3ys[i])/(2Δ)
disc_diff2_d(i, ys = ys) = (ys[i-2]-2*ys[i-1]+ys[i])/(Δ^2)
i1, i2 = 3,4

eqs = [
    quintic_pol.subs(x,xs[i1]) - ys[i1], # f(0)
    quintic_pol.subs(x,xs[i2]) - ys[i2], # f(Δ)
    quintic_pol.diff(x).subs(x,xs[i1]) - disc_diff_c(i1,ys),# f'(0)
    quintic_pol.diff(x).subs(x,xs[i2]) - disc_diff_d(i2,ys), # f'(Δ)
    quintic_pol.diff(x,2).subs(x,xs[i1]) - disc_diff2_c(i1,ys),# f''(0)
    quintic_pol.diff(x,2).subs(x,xs[i2]) - disc_diff2_d(i2,ys), # f''(Δ)
]
eqs = expand.(eqs)

sol = solve(eqs,cs)
appr = quintic_pol.subs(sol) |> expand

for y in ys
    appr.coeff(y).subs(x/Δ,δ) |> display
end

   5      4         
  δ    4⋅δ     3   δ
- ── + ──── - δ  + ─
  2     3          6

   5                  2    
3⋅δ       4      3   δ     
──── - 4⋅δ  + 3⋅δ  + ── - δ
 2                   2     

     5                           
  3⋅δ       4      3    2   δ    
- ──── + 4⋅δ  - 3⋅δ  - δ  + ─ + 1
   2                        2    

 5      4         2    
δ    4⋅δ     3   δ    δ
── - ──── + δ  + ── + ─
2     3          2    3

0

0

### Omitting Boundary conditions

In [4]:
disc_diff_0(i, ys = ys) = (- 2*ys[i-1] - 3*ys[i] +6*ys[i+1] - ys[i+2])/(6Δ)
disc_diff2_0(i, ys = ys) = (ys[i-1]-2*ys[i]+*ys[i+1])/(Δ^2)
i1, i2 = 3,4

cubic_pol = sum(symbols("c_{$i}")*x^i for i in 0:3)
eqs = [
    cubic_pol.subs(x,xs[2]) - ys[2], # f(0)
    cubic_pol.subs(x,xs[3]) - ys[3], # f(Δ)
    cubic_pol.diff(x).subs(x,xs[3]) - disc_diff(3,ys), # f'(Δ)
    cubic_pol.diff(x,2).subs(x,xs[3]) - disc_diff2(3,ys), # f''(Δ)
]
eqs = expand.(eqs)

sol = solve(eqs,cs)
appr = cubic_pol.subs(sol) |> expand

for y in ys
    appr.coeff(y).subs(x/Δ,δ) |> display
end

0

 2          
δ    3⋅δ    
── - ─── + 1
2     2     

   2      
- δ  + 2⋅δ

 2    
δ    δ
── - ─
2    2

In [7]:
# End: not enforcing unknown conditions
cubic_pol = sum(symbols("c_{$i}")*x^i for i in 0:3)
eqs = [
    cubic_pol.subs(x,xs[2]) - ys[2], # f(0)
    cubic_pol.subs(x,xs[3]) - ys[3], # f(Δ)
    cubic_pol.diff(x).subs(x,xs[2]) - disc_diff(2,ys),# f'(0)
    cubic_pol.diff(x,2).subs(x,xs[2]) - disc_diff2(2,ys),# f''(0)
]
eqs = expand.(eqs)

sol = solve(eqs, cs)
appr = cubic_pol.subs(sol) |> expand

for y in ys
    appr.coeff(y).subs(x/Δ,δ) |> display
end

 2    
δ    δ
── - ─
2    2

     2
1 - δ 

 2    
δ    δ
── + ─
2    2

0